In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [50]:
# Read in the data

df = pd.read_csv("C:\\Users\\joaju\\OneDrive\\Projects\\Mushroom_Classifier\\mushroom_cleaned.csv")

In [51]:
X = pd.get_dummies(df.drop(['class'], axis = 1))
y = df['class']

In [52]:
# Outputs of the train_test_split function, so that training from X and y the result is stored in the four variables.

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [53]:
X_train.head()

,cap-diameter,cap-shape,gill-attachment,gill-color,stem-height,stem-width,stem-color,season
23451,854,6,6,10,0.473074,2064,11,0.943195
39876,636,4,0,7,3.298156,1594,11,0.943195
19047,586,5,1,0,0.278955,1229,11,1.804273
12357,29,6,1,6,0.569421,48,7,0.888450
37932,572,0,1,11,0.127753,635,12,0.888450


In [54]:
y_train.head()

23451    0
39876    1
19047    0
12357    1
37932    1
Name: class, dtype: int64

In [55]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

Build our Model by adding layers

In [56]:
model = Sequential()

# Add layers to the neural networks
# units : how many neurons in the dense layer
# activation function
model.add(Dense(units=32, activation='relu', input_dim=len(X_train.columns)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

c:\Users\joaju\anaconda4\envs\newenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Compile our Model

In [57]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

I have detected that our model does not increase in accuracy from 200 to 500 epochs.
So we will apply early stopping.

In [58]:
from tensorflow.keras.callbacks import EarlyStopping

In [59]:
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Retrain the model applying the early stopping
history = model.fit(X_train, y_train, epochs=200, 
                    batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/200
1081/1081 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5505 - loss: 9.0209 - val_accuracy: 0.5456 - val_loss: 0.6704
Epoch 2/200
1081/1081 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5503 - loss: 0.6724 - val_accuracy: 0.5456 - val_loss: 0.6692
Epoch 3/200
1081/1081 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5510 - loss: 0.6919 - val_accuracy: 0.5456 - val_loss: 0.6725
Epoch 4/200
1081/1081 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5519 - loss: 0.6713 - val_accuracy: 0.5456 - val_loss: 0.6726
Epoch 5/200
1081/1081 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5456 - loss: 0.6734 - val_accuracy: 0.5456 - val_loss: 0.6730
Epoch 6/200
1081/1081 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5458 - loss: 0.6736 - val_accuracy: 0.5456 - val_loss: 0.6731
Epoch 7/200
1081/1081 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5536 - loss: 0.6711 - val_accuracy: 0.5456 - val_loss: 0.6732
Epoch 8/200
1081/1081 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5560 - loss: 0

Re-evaluate the Early Stopping Model

In [64]:
y_hat = model.predict(X_test)
y_hat = [0 if val < 0.5 else 1 for val in y_hat]
accuracy_score(y_test, y_hat)

338/338 ━━━━━━━━━━━━━━━━━━━━ 0s 820us/step


0.5469603035069862

Save and Load our Model

In [65]:
# Saving our model in a file or directory so that it saves
# the training configuration such as optimizer, loss and metrics.
# it can be loaded and used later without having to retrain it.

model.save('tfmodel.keras')

In [66]:
# Delete the model from the memory in this environment.

del model

In [67]:
# Later on load_model function will load our saved model as follows.

model = load_model('tfmodel.keras')